# Práctica sobre algoritmos genéticos

<img src="imgs/tren.jpg">


Un área ferroviaria de carga/descarga con una única vía de entrada y otra salida se compone de tres muelles de carga/descarga: Op1, Op2 y Op3, correspondientes a contenedores, carbón y gas. Por tanto, cada tren que llega se dirige a un muelle en función de su carga. Un tren tarda en cargar o descargar un tiempo proporcional al número de vagones que arrastra. Cada día llegan secuencialmente n trenes. Si los trenes son de cargas distintas, pueden entrar en paralelo a los muelles. Cuando dos trenes con el mismo tipo de carga se encuentran seguidos, el segundo debe esperar por el primero, así como todos los demás que se encuentren por detrás.


Se nos plantea resolver, mediante un algoritmo genético, el problema de la ordenación en la entrada de los trenes para minimizar el tiempo de paso del conjunto de trenes.


<img src="imgs/docks.jpg" width="70%">

In [1]:
import random
import numpy
from deap import base, creator, tools, algorithms

## Trenes

En el siguiente código, <code>random_trains_generation</code> genera los trenes que llegarán en un día. Cada ordenación diferente de este conjunto corresponderá con un **individuo** en nuestro eaquema de algoritmo genético.

In [2]:
import random
# from deap import base, creator, tools

class Train:
    def __init__(self, wagons, op, licence_plate):
        self.wagons = wagons
        self.op = op
        self.licence_plate = licence_plate

    def __str__(self):
        return "Número de vagones:" + str(self.wagons) \
        + "\n" + "Muelle de operaciones:" + str(self.op) \
        + "\n" + "Matrícula:" + str(self.licence_plate)
    
    def __repr__(self):
        return f"Train({self.wagons}, '{self.op}', {self.licence_plate})"

def random_trains_generation(n):

    train_list = []
    
    for i in range(n):
        wagons = random.randint(10, 30)  # Cada tren puede arrastrar entre 10 y 30 vagones
        op = random.choice(["op1", "op2", "op3"])  # A cada tren se le asigna un tipo de carga
        train_list.append(Train(wagons, op, i))

    return train_list


In [3]:
trains = random_trains_generation(25)

print(trains[7])
trains[7]

Número de vagones:17
Muelle de operaciones:op2
Matrícula:7


Train(17, 'op2', 7)

# Solución

In [4]:
from collections import defaultdict

def trains_cost(indices, trains):
    loaders = defaultdict(lambda: 0)
    time = 0
    for i in indices:
        train = trains[i]
        time = max(time, loaders[train.op])
        loaders[train.op] = time + train.wagons
    return max(*loaders.values()),


# Usando DEAP

In [5]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

In [7]:
def shuffle_trains(size):
    ind = creator.Individual(range(size))
    random.shuffle(ind)
    return ind

In [85]:
import numpy as np

TRAIN_LEN = 25

toolbox = base.Toolbox()
all_trains = random_trains_generation(TRAIN_LEN)
toolbox.register("individual", shuffle_trains, TRAIN_LEN)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", trains_cost, trains=all_trains)
toolbox.register("mate", tools.cxOrdered)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

pop = toolbox.population(n=100)

In [96]:
hof = tools.HallOfFame(4)

stats = tools.Statistics(lambda indiv: indiv.fitness.values)
stats.register("avg", np.mean)
stats.register("min", np.min)
stats.register("max", np.max)

pop, logbook = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=100, stats=stats, halloffame=hof, verbose=True)


gen	nevals	avg   	min	max
0  	0     	202.77	198	255
1  	59    	204.03	198	288
2  	52    	201.61	198	250
3  	62    	203.41	198	275
4  	53    	202.61	198	288
5  	61    	201.22	198	251
6  	59    	202.07	198	256
7  	57    	200.83	198	259
8  	54    	200.62	198	244
9  	64    	200.77	198	264
10 	62    	201.15	198	250
11 	57    	203.13	198	252
12 	69    	202.83	198	250
13 	66    	202.02	198	281
14 	57    	203.22	198	268
15 	61    	203.9 	198	256
16 	58    	202.89	198	258
17 	66    	200.47	198	246
18 	69    	205.1 	198	278
19 	60    	202.75	198	253
20 	64    	202.31	198	264
21 	67    	203.77	198	263
22 	59    	202.11	198	266
23 	63    	202.05	198	252
24 	66    	204.77	198	272
25 	60    	202.81	198	278
26 	63    	201.17	198	253
27 	65    	202.13	198	253
28 	59    	201.68	198	260
29 	55    	201.57	198	240
30 	52    	202.67	198	268
31 	59    	204.45	198	281
32 	61    	200.44	198	259
33 	69    	201.76	198	251
34 	63    	203.84	198	286
35 	67    	204.15	198	265
36 	55    	202.81	198	271
37 	52    	2

In [106]:
for i in range(4):
		print(hof[i], hof[i].fitness.values)

[9, 16, 7, 19, 20, 10, 2, 22, 4, 18, 24, 0, 1, 21, 11, 13, 6, 3, 5, 15, 14, 12, 23, 17, 8] (198.0,)
[16, 9, 7, 19, 20, 10, 2, 22, 18, 4, 24, 0, 1, 21, 11, 13, 6, 3, 5, 15, 14, 12, 23, 17, 8] (198.0,)
[9, 16, 7, 19, 20, 10, 2, 22, 18, 4, 24, 0, 1, 21, 11, 13, 6, 3, 5, 15, 14, 12, 23, 17, 8] (198.0,)
[17, 16, 7, 19, 20, 10, 2, 22, 4, 18, 24, 0, 1, 21, 11, 13, 6, 3, 5, 15, 14, 12, 23, 8, 9] (198.0,)


In [151]:
from PIL import Image, ImageDraw, ImageFont

width, height = 1920, 1080

def animate_trains(trains):
	images = []
	timer = 0
	loaders = {'op1':None,'op2': None, 'op3': None}
	for train in trains:
		img = Image.new('RGB', (width, height), (180, 180, 180))
		draw = ImageDraw.Draw(img)
		# get a 50 px font
		fnt = ImageFont.truetype('BebasNeue-Regular.ttf', 50)
		#draw.text((50, 50), "howdy", font=fnt)
		lower_track = 0.9 * height
		upper_track = 0.1*height
		draw.line((0, lower_track, 1100, lower_track), fill=(0, 0, 0), width=10)
		ops = ["op1", "op2", "op3"]
		pos = [700, 900, 1100]
		for p, op in zip(pos, ops):
			draw.line((p, lower_track, p+500, upper_track), fill=(0, 0, 0), width=10)
			draw.text((p+150, lower_track-250), op, font=fnt, fill=(0, 0, 0))
		draw.line((1200, upper_track, width, upper_track), fill=(0, 0, 0), width=10)

		draw.text((40, 40), "Timer: 000", fill=(255, 255, 255), font=fnt)
		images.append(img)
	
	images[0].save("train.gif", format="GIF", append_images=images[1:], save_all=True)

In [152]:
trains = [all_trains[i] for i in hof[0]]
animate_trains(trains)